[URL dashboard](https://public.tableau.com/app/profile/jason.rich.darmawan/viz/h8dsft_Milestone1_jason_rich_darmawan_onggo_putra/StorytellingTime?publish=yes)

# Perkenalan

Nama : Jason Rich Darmawan Onggo Putra

Batch : 016 RMT

# Identifikasi Masalah

## Topik Permasalahan

Perbandingan Pembelian Alkohol di Iowa, Amerika Serikat sebelum Pandemi Covid-19 dan puncak (kasus baru per hari) Pandemi Covid-19

Data sebelum Pandemi Covid-19 yang digunakan adalah bulan Januari 2018.

Data Puncak Pandemi Covid-19 yang digunakan adalah bulan Januari 2021.

## Problem Statement

Mengetahui Kota, Ukuran Botol, Kategori Minuman Beralkohol yang harus distok dan potensi nilai pasar yang dilayani ketika terjadi krisis.

## Latar Belakang

Sebagai Data Analyst yang mungkin akan bekerja di e-commerce atau franchisor convenience store, saya perlu intuisi untuk "apa yang harus dilakukan ketika terjadi krisis seperti Pandemi Covid-19", "berapa nilai GMV minuman beralkohol jika membuka toko di kota Top 1, dengan ukuran botol Top 1-5, dan kategori minuman beralkohol Top 1-5"

## Penjabaran Masalah

### Visualisasi

Tujuan: mencari kota terbaik untuk menjual minuman beralkohol

- [x] Di kota (column `city`) mana penjualan tertinggi (dalam volume / column `volume_sold_liters`) minuman beralkohol di Iowa, Amerika Serikat pada bulan Januari 2018 dan 2022?
- [x] Di kota (column `city`) mana penjualan tertinggi (dalam total harga / column `sale_dollars`) di Iowa, Amerika Serikat pada bulan Januari 2018 dan 2022?

Tujuan: menentukan ukuran botol dan kategori minuman beralkohol yang harus distok

- [x] Apa ukuran botol (column `bottle_volume_ml`) minuman beralkohol yang paling banyak terjual (column `bottles_sold`) di Iowa, Amerika Serikat pada bulan Januari 2018 dan 2022?
- [x] Apa kategori (column `category_name`) minuman beralkohol yang paling banyak terjual (column `bottles_sold`) di Iowa, Amerika Serikat pada bulan Januari 2018 dan 2022?

### Statistik Deskriptif

Tujuan: membuat promo **Min. Jumlah Volume** dan **Min. Belanja**

- [x] Berapa rata-rata penjualan minuman beralkohol (dalam volume / column `volume_sold_liters`) di Iowa, Amerika Serikat pada bulan Januari 2018 dan 2022?
- [x] Berapa rata-rata penjualan minuman beralkohol (dalam total harga / column `sale_dollars`) di Iowa, Amerika Serikat pada bulan Januari 2018 dan 2022?

Tujuan: mengetahui nilai GMV minuman beralkohol di Iowa, Amerika Serikat

- [x] Berapa GMV minuman beralkohol di Iowa, Amerika Serikat pada bulan Januari 2018 dan 2022?

### Hypothesis Testing 1

- [x] Apa rata-rata penjualan minuman beralkohol (dalam volume / column `volume_sold_liters`) di Iowa, Amerika Serikat pada bulan Januari 2018 dan 2022 sama/berbeda (lebih kecil/lebih besar)?
- [x] Apa rata-rata penjualan minuman beralkohol (dalam total harga / column `sale_dollars`) di Iowa, Amerika Serikat pada bulan Januari 2018 dan 2022 sama/berbeda?

### Hypothesis Testing 2

- [x] Apa rata-rata harga minuman beralkohol (column `state_bottle_retail`) di Iowa, Amerika Serikat pada bulan Januari 2018 dan 2022 sama/berbeda?

# Data Loading

## Import Library

In [ ]:
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# fix plotly is not showing in github.
# if you want to have interactive plotly, use `fig.show("notebook_connected")`
import plotly.io as pio
pio.renderers.default = "svg"

from scipy import stats

## Reference

project: bigquery-public-data

dataset: iowa_liquor_sales

table: sales

## Query

advantage: sub-query agar lebih clean dan lebih mudah ganti tahun dan bulan.

disadvantage: result sub-query-nya boros (diload hanya untuk difilter di query berikutnya).

```
SELECT *,
FROM (
  SELECT *, 
         EXTRACT(YEAR FROM date) as year,
         EXTRACT(MONTH FROM date) as month
  FROM `bigquery-public-data.iowa_liquor_sales.sales`
  WHERE date BETWEEN "2018-01-01" AND "2022-01-31"
)
WHERE year IN (2018, 2022) 
 AND month=1
```

In [ ]:
df = pd.read_csv("./bigquery-public-data_iowa_liquor_sales_year_2018_and_year_2022_month_1.csv")

## DataFrame.head()

drop columns
- [x] can't be visualized nor answer 'Penjabaran Masalah'
  `store_number`, `store_name`, `address`, `store_location`, `vendor_number`, `vendor_name`, `item_number`, `item_description`
- too big to derive conclusion from
  `county_number`, `county`
- duplicated columns
  `volume_sold_gallons`

DataFrame.sort_values() 
- [x] by column `date`

In [ ]:
pd.options.display.max_columns = None
df.head()

In [ ]:
# Is column invoice_and_item_number value unique?
df.loc[df['invoice_and_item_number'].map(df['invoice_and_item_number'].value_counts()) > 1]
# alternative method:
# df.loc[df['invoice_and_item_number'].duplicated()]

## DataFrame.columns

In [ ]:
df.columns

## Column `category_name` Values

In [ ]:
df['category'].value_counts().index

## Column `city` Values

In [ ]:
df['city'].value_counts().index

## Column `bottle_volume_ml` Values

In [ ]:
df['bottle_volume_ml'].value_counts().index.sort_values()

## DataFrame.describe()

detecting and treating outliers on columns

- [x] year 2018
  - columns 'pack', 'bottle_volume_ml', 'state_bottle_cost', 'state_bottle_retail', 'bottles_sold', 'sale_dollars', 'volume_sold_liters'
- [x] year 2022
  - columns 'pack', 'bottle_volume_ml', 'state_bottle_cost', 'state_bottle_retail', 'bottles_sold', 'sale_dollars', 'volume_sold_liters'

In [ ]:
def calculateLowerLimitUpperLimit(series: pd.Series):
    skew = series.skew()

    # extreme skewed distribution
    if skew < -1 or skew > 1:
        q1, q3 = series.quantile([.25,.75])
        iqr = q3 - q1
        return q1 - iqr * 3, q3 + iqr * 3
        
    # skewed distribution
    elif skew < -0.5 or skew > 0.5:
        q1, q3 = series.quantile([.25,.75])
        iqr = q3 - q1
        return q1 - iqr * 1.5, q3 + iqr * 1.5

    # normal distribution
    else:
        mean, std = series.agg(['mean', 'std'])
        return mean - 3 * std, mean + 3 * std

def detect_lower_limit_min_upper_limit_max():
    for year in [2018, 2022]:
        for index, value in df.loc[(df['year'] == year) & (df['month'] == 1), 
                                   ['pack', 'bottle_volume_ml', 'state_bottle_cost', 
                                    'state_bottle_retail', 'bottles_sold', 'sale_dollars', 
                                    'volume_sold_liters']] \
                              .items():
            lowerLimit, upperLimit = calculateLowerLimitUpperLimit(value)

            min, max = value.agg(['min', 'max'])
            if lowerLimit > min or upperLimit < max:
                print("year {0}, column {1}, lower limit {2}, min {3}, upper limit {4}, max {5}" \
                      .format(year, index, lowerLimit, min, upperLimit, max))

detect_lower_limit_min_upper_limit_max()

## DataFrame.info()

handle null rows on columns

range index: 373231 entries
- [x] remove null due to categorical data
  - `city`            372256
  - `zip_code`        372256
  - `category`        373094
  - `category_name`   373094

In [ ]:
df.info()

# Data Cleaning

In [ ]:
df2 = df.sort_values('date').reset_index(drop=True).copy()

In [ ]:
df3 = df2.drop(['store_number', 'store_name', 'address', 'store_location', 
                'vendor_number', 'vendor_name', 'item_number', 'item_description', 
                'county_number', 'county', 'volume_sold_gallons'], 
                axis='columns') \
         .copy()

## Outliers Detection and Treatment

In [ ]:
def detectOutliers(series: pd.Series, year: int):
    lowerLimit, upperLimit = calculateLowerLimitUpperLimit(series)
    outliersPercentage = len(series.loc[(series < lowerLimit) | (series > upperLimit)]) \
                        / len(series)
    print("year {0}, column {1}, skew {2:.2f}, outliers percentage {3:.2f}%".format(year, series.name, series.skew(), outliersPercentage * 100))
    print("lower limit {0[0]}, upper limit {0[1]}".format(calculateLowerLimitUpperLimit(series)))
    display(series.value_counts().sort_index(ascending=False))

- [x] year 2018
  - natural outliers
    - under 5%
      - 'pack', 'bottle_volume'
  - not natural outliers
    - under 5%
      - 'state_bottle_cost', 'state_bottle_retail', 'bottles_sold', 'sale_dollars', 'volume_sold_liters'
- [x] year 2022
  - natural outliers
    - under 5%
      - 'pack', 'bottle_volume_ml'
  - not natural outliers
    - under 5%
      - 'state_bottle_cost', 'state_bottle_retail', 'bottles_sold', 'sale_dollars', 'volume_sold_liters'

In [ ]:
def detect_outliers():
    for year in [2018, 2022]:
        for index, value in df3.loc[(df3['year'] == year) & (df3['month'] == 1), 
                                    ['pack', 'bottle_volume_ml', 'state_bottle_cost', 
                                    'state_bottle_retail', 'bottles_sold', 'sale_dollars', 
                                    'volume_sold_liters']] \
                            .items():
            detectOutliers(value, year)

detect_outliers()

### Ask Instructor

reference: https://www.quora.com/Is-it-bad-to-delete-statistical-outliers

- [x] Is it true in Machine Learning, it is always a requirement to remove the outliers? If so, should we remove natural outliers or do mean/median imputation in column `bottle_volume_ml`?
  - do not alter natural outliers.

In [ ]:
df4 = df3.copy()

def treat_outliers(dataframe: pd.DataFrame):
    # outliers percentage under 5% and not natural outliers
    for year in [2018, 2022]:
        #WARNING: do not edit the DataFrame per column. 
        # gather all outliers index. Then drop it with one move.
        index = pd.Index([])
        
        for column in ['state_bottle_cost', 'state_bottle_retail', 
                    'bottles_sold', 'sale_dollars', 'volume_sold_liters']:
            value = dataframe.loc[(dataframe['year'] == year) & (dataframe['month'] == 1), column]
            lowerLimit, upperLimit = calculateLowerLimitUpperLimit(value)
            index = index.append(value[(value < lowerLimit) | (value > upperLimit)].index)
        
        dataframe = dataframe.drop(index)
    
    return dataframe

df4 = treat_outliers(df4)

### Ask Instructor

- [x] after 1st outliers treatment, what should we do if the outliers percentage still above 0.3%?
  - it's okay.

In [ ]:
def detect_outliers_again():
    for year in [2018, 2022]:
        for index, value in df4.loc[(df4['year'] == year) & (df4['month'] == 1), 
                                    ['pack', 'bottle_volume_ml', 'state_bottle_cost', 
                                    'state_bottle_retail', 'bottles_sold', 'sale_dollars', 
                                    'volume_sold_liters']] \
                            .items():
            detectOutliers(value, year)

detect_outliers_again()

## Null Treatment

### Ask Instructor

- [x] should we remove rows based on condition (a column with categorical data is null)?
  - it's okay to drop it.

In [ ]:
def detect_null_percentage():
    for index, value in df4.loc[:,['city','zip_code','category','category_name']].items():
        print("column {0}, null percentage {1:.2f}%".format(index, len(value.loc[value.isna()]) / len(value) * 100))

detect_null_percentage()

In [ ]:
df5 = df4.copy()

def treat_null(dataframe: pd.DataFrame):
    # categorical data, we will drop them.
    for column in ['city', 'zip_code', 'category', 'category_name']:
        series = dataframe.loc[dataframe[column].isna(), column]
        dataframe = dataframe.drop(series.index)
    
    return dataframe

df5 = treat_null(df5)

In [ ]:
df5.info()

In [ ]:
df6 = df5.reset_index()

In [ ]:
df6.to_csv("bigquery-public-data_iowa_liquor_sales_year_2018_and_year_2022_month_1_cleaned.csv")

# Analisis dan Perhitungan

## Visualisasi Data

### Des Mones adalah kota dengan penjualan tertinggi minuman beralkohol di Iowa, Amerika Serikat pada bulan Januari 2018 dan 2022

In [ ]:
def visual_top_5_cities_by_volume_sold_liters_or_sale_dollars():
    for column in ["volume_sold_liters", "sale_dollars"]:
        fig = make_subplots(rows=1, cols=2,
                            specs=[[{"type": "pie"}, {"type": "pie"}]])

        for index, year in enumerate([2018, 2022]):
            dataframe = df6.loc[(df6['year'] == year) & (df6['month'] == 1)]
            series_other = dataframe.groupby('city')[column] \
                                    .sum() \
                                    .sort_values(ascending=False) \
                                    .index[5:]
            series = dataframe.replace(series_other, 'Other') \
                            .groupby('city')[column] \
                            .sum()
            fig.add_trace(go.Pie(values=series,
                                labels=series.index,
                                title="Top 5 cities by {0} in January {1}".format(column, year)),
                        row=1, col=index+1)
        
        fig.show()

visual_top_5_cities_by_volume_sold_liters_or_sale_dollars()

### 750 ml (Januari 2018 & 2022) adalah ukuran botol minuman beralkohol yang paling banyak terjual.

Dengan menyediakan ukuran botol minuman beralkohol 750ml, perusahaan dapat melayani 43.8% (Januari 2018) dan 44.3% (Januari 2022) pasar.

Untuk menciptakan "illusion of choice" dalam convenience store, perusahaan dapat menyediakan ukuran botol minuman beralkohol 750 ml, 1750 ml, 375 ml, 1000 ml untuk melayani 91.95% (2018) dan 89.23% (2022) pasar.

In [ ]:
def visual_top_5_bottles_sold_by_bottle_volume_ml():
    for group in ['bottle_volume_ml']:
        fig = make_subplots(rows=1, cols=2,
                            specs=[[{"type": "pie"}, {"type": "pie"}]])
        for index, year in enumerate([2018, 2022]):
            # .copy() because we are going to change column type.
            dataframe = df6.loc[(df6['year'] == year) & (df6['month'] == 1)].copy()
            dataframe['bottle_volume_ml'] = dataframe[group].astype(str)

            series_other = dataframe.groupby(group)['bottles_sold'] \
                                    .sum() \
                                    .sort_values(ascending=False) \
                                    .index[5:]

            series = dataframe.replace(series_other, 'Other') \
                            .groupby(group)['bottles_sold'] \
                            .sum()
                            
            fig.add_trace(go.Pie(values=series,
                                labels=series.index,
                                title="Top 5 bottles sold by {0} in January {1}".format(group, year)),
                        row=1, col=index+1)

        fig.show()

visual_top_5_bottles_sold_by_bottle_volume_ml()

### American Vodkas (Januari 2018 & 22) adalah kategori minuman beralkohol paling banyak terjual.

Dengan hanya menyediakan kategori minuman beralkohol American Vodkas, perusahana dapat melayani 20.6% (2018) dan 20.3% (2022) pasar pada masing-masing tahun.

Untuk menciptakan "illusion of choice" dalam convenience store, perusahaan dapat menyediakan kategori minuman beralkohol American Vodkas, Canadian Whiskies, Spiced Rum, Straight Bourbon Whiskies, Whiskey Liquer untuk melayani 48.5% (Januari 2018) dan 49.2% (Januari 2022) pasar.

In [ ]:
def visual_top_5_bottles_sold_by_category_name():
    for group in ['category_name']:
        fig = make_subplots(rows=1, cols=2,
                            specs=[[{"type": "pie"}, {"type": "pie"}]])
        for index, year in enumerate([2018, 2022]):
            # .copy() because we are going to change column type.
            dataframe = df6.loc[(df6['year'] == year) & (df6['month'] == 1)].copy()
            dataframe['bottle_volume_ml'] = dataframe[group].astype(str)

            series_other = dataframe.groupby(group)['bottles_sold'] \
                                    .sum() \
                                    .sort_values(ascending=False) \
                                    .index[5:]

            series = dataframe.replace(series_other, 'Other') \
                            .groupby(group)['bottles_sold'] \
                            .sum()
                            
            fig.add_trace(go.Pie(values=series,
                                labels=series.index,
                                title="Top 5 bottles sold by {0} in January {1}".format(group, year)),
                        row=1, col=index+1)

        fig.show()

visual_top_5_bottles_sold_by_category_name()

## Statistik Deskriptif

sub-bab sebelumnya termasuk Statistik Deskriptif. sub-bab ini dipisah untuk memudahkan penilaian sesuai assignment rubics.

### 6.15L (Januari 2018) dan 6.08L (Januari 2022) adalah rata-rata penjualan minuman beralkohol per transaksi di Iowa, Amerika Serikat.

In [ ]:
def visual_volume_sold_liters_mean():
    series = df6.groupby(['year', 'month'])['volume_sold_liters'].mean()
    series.index = ['{1} {0}'.format(i, j) for i, j in series.index]
    px.bar(series).show()

visual_volume_sold_liters_mean()

### \$89.26 (Januari 2018) dan \$106.83 (Januari 2022) adalah rata-rata penjualan minuman beralkohol per transaksi di Iowa, Amerika Serikat.

In [ ]:
def visual_sale_dollars_mean():
    series = df6.groupby(['year', 'month'])['sale_dollars'].mean()
    series.index = ['{1} {0}'.format(i, j) for i, j in series.index]
    px.bar(series).show()

visual_sale_dollars_mean()

### \$16.04 million (Januari 2018) dan \$18.09 million (Januari 2022) adalah GMV minuman beralkohol.

In [ ]:
def visual_sale_dollars_sum():
    series = df6.groupby(['year', 'month'])['sale_dollars'].sum()
    series.index = ['{1} {0}'.format(i, j) for i, j in series.index]
    px.bar(series).show()

visual_sale_dollars_sum()

## Statistik Inferensial

Terdapat banyak faktor yang berbeda (hari libur, pandemi, dan lain-lain) dalam masing-masing data (2018 dan 2022). Faktor ini menyebabkan kita tidak dapat membandingkan kedua data secara adil.

Oleh karena itu, kita menggunakan hypothesis testing two sample untuk menguji apakah kedua data tersebut sama atau berbeda dari sisi:

1. rata-rata penjualan dalam volume, total harga pada bulan Januari 2018 dan 2021.
2. rata-rata harga minuman pada bulan Januari 2018 dan 2021.

### Rata-rata penjualan minuman beralkohol dalam volume pada bulan Januari 2018 dan 2022 berbeda.

H0: μ(2018, volume_sold_liters) = μ(2022, volume_sold_liters)

H1: μ(2018, volume_sold_liters) != μ(2022, volume_sold_liters)

We will reject null hypothesis if p-value is below 1%.

Since our p-value is less than 0.5%, so we reject the null hypothesis and we can conclude with 99% confidence that, the average sales in terms of volume sold liters in January 2018 and January 2022 is different.

In [ ]:
def hypothesis_testing_two_sample_volume_sold_liters_unequal():
    series_1_2018 = df6.loc[(df6['year'] == 2018) & (df6['month'] == 1),'volume_sold_liters']
    series_1_2022 = df6.loc[(df6['year'] == 2022) & (df6['month'] == 1),'volume_sold_liters']

    t_stat, p_val = stats.ttest_ind(series_1_2018, series_1_2022)
    print("p-value {:.2f}".format(p_val))

hypothesis_testing_two_sample_volume_sold_liters_unequal()

#### Rata-rata penjualan minuman beralkohol dalam volume pada bulan Januari 2018 lebih besar daripada Januari 2022

H0: μ(2018, volume_sold_liters) >= μ(2022, volume_sold_liters)

H1: μ(2018, volume_sold_liters) < μ(2022, volume_sold_liters)

We will reject null hypothesis if p-value is below 0.5%.

Based on the result below, we can conclude with 99% confidence that we fail to reject the null hypothesis which means the average sales in terms of volume sold liters in January 2018 is more than equal compared to January 2022.

In [ ]:
def hypothesis_testing_two_sample_volume_sold_liters_less():
    series_1_2018 = df6.loc[(df6['year'] == 2018) & (df6['month'] == 1),'volume_sold_liters']
    series_1_2022 = df6.loc[(df6['year'] == 2022) & (df6['month'] == 1),'volume_sold_liters']

    t_stat, p_val = stats.ttest_ind(series_1_2018, series_1_2022, alternative="less")
    print("p-value {:.2f}".format(p_val))

hypothesis_testing_two_sample_volume_sold_liters_less()

### Rata-rata penjualan minuman beralkohol per transaksi pada bulan Januari 2018 dan 2022 berbeda

H0: μ(2018, sale_dollars) = μ(2022, sale_dollars)

H1: μ(2018, sale_dollars) != μ(2022, sale_dollars)

We will reject null hypothesis if p-value is below 0.5%.

Since our p-value is less than 0.5%, so we reject the null hypothesis and we can conclude with 99% confidence that, the average sales in terms of dollars in January 2018 and January 2022 is different.

In [ ]:
def hypothesis_testing_two_sample_sale_dollars_unequal():
    series_1_2018 = df6.loc[(df6['year'] == 2018) & (df6['month'] == 1),'sale_dollars']
    series_1_2022 = df6.loc[(df6['year'] == 2022) & (df6['month'] == 1),'sale_dollars']

    t_stat, p_val = stats.ttest_ind(series_1_2018, series_1_2022)
    print("p-value {:.2f}".format(p_val))

hypothesis_testing_two_sample_sale_dollars_unequal()

#### Rata-rata penjualan minuman beralkohol per transaksi pada bulan Januari 2018 lebih kecil daripada 2022.

H0: μ(2018, sale_dollars) <= μ(2022, sale_dollars)

H1: μ(2018, sale_dollars) > μ(2022, sale_dollars)

We will reject null hypothesis if p-value is below 0.5%.

Based on the result below, we can conclude with 99% confidence that we fail to reject the null hypothesis which means the average sales in terms of dollars in January 2018 is less than equal compared to January 2022.

In [ ]:
def hypothesis_testing_two_sample_sale_dollars_greater():
    series_1_2018 = df6.loc[(df6['year'] == 2018) & (df6['month'] == 1),'sale_dollars']
    series_1_2022 = df6.loc[(df6['year'] == 2022) & (df6['month'] == 1),'sale_dollars']

    t_stat, p_val = stats.ttest_ind(series_1_2018, series_1_2022, alternative="greater")
    print("p-value {:.2f}".format(p_val))

hypothesis_testing_two_sample_sale_dollars_greater()

### Rata-rata harga minuman beralkohol pada bulan Januari 2018 dan 2022 berbeda

H0: μ(2018, state_bottle_retail) = μ(2022, state_bottle_retail)

H1: μ(2018, state_bottle_retail) != μ(2022, state_bottle_retail)

We will reject null hypothesis if p-value is below 1%.

Since our p-value is less than 1%, so we reject the null hypothesis and we can conclude with 99% confidence that, the average bottle retail price in January 2018 and January 2022 is different.

In [ ]:
def hypothesis_testing_two_sample_state_bottle_retail_unequal():
    series_1_2018 = df6.loc[(df6['year'] == 2018) & (df6['month'] == 1), 'state_bottle_retail']
    series_1_2022 = df6.loc[(df6['year'] == 2022) & (df6['month'] == 1), 'state_bottle_retail']

    t_stat, p_val = stats.ttest_ind(series_1_2018, series_1_2022)
    print("p-value {:.2f}".format(p_val))

hypothesis_testing_two_sample_state_bottle_retail_unequal()

#### Rata-rata harga minuman beralkohol pada bulan Januari 2018 lebih rendah daripada 2022

H0: μ(2018, state_bottle_retail) >= μ(2022, state_bottle_retail)

H1: μ(2018, state_bottle_retail) < μ(2022, state_bottle_retail)

We will reject null hypothesis if p-value is below 0.5%.

Since our p-value is less than 0.5, so we reject the null hypothesis and we can conclude with 99% confidence that, the average bottle retail price in January 2018 is less than equal compared to January 2022.

In [ ]:
def hypothesis_testing_two_sample_state_bottle_retail_less():
    series_1_2018 = df6.loc[(df6['year'] == 2018) & (df6['month'] == 1), 'state_bottle_retail']
    series_1_2022 = df6.loc[(df6['year'] == 2022) & (df6['month'] == 1), 'state_bottle_retail']

    t_stat, p_val = stats.ttest_ind(series_1_2018, series_1_2022, alternative='less')
    print("p-value {:.2f}".format(p_val))

hypothesis_testing_two_sample_state_bottle_retail_unequal()

# Pengambilan Keputusan

## The Insights

1. Des Mones adalah kota dengan penjualan tertinggi minuman beralkohol di Iowa, Amerika Serikat pada bulan Januari 2018 dan 2022
2. 750 ml (2018, 2022) adalah ukuran botol minuman beralkohol yang paling banyak terjual pada masing-masing tahun.
   1. 91.95% (Januari 2018) dan 89.23% (Januari 2022) pasar dapat dilayani dengan hanya menyediakan ukuran botol minuman beralkohol 750 ml, 1750 ml, 375 ml, dan 1000 ml.
3. American Vodkas (Januari 2018 & 2022) adalah kategori minuman beralkohol yang paling banyak terjual.
   1. 48.5% (Januari 2018) dan 49.2% (Januari 2022) pasar dapat dilayani dengan hanya 5 kategori minuman, yaitu American Vodkas, Canadian Whiskies, Spiced Rum, Straight Bourbon Whiskies, dan Whiskey Liquer.
4. 6.15L (Januari 2018) dan 6.08L (Januari 2022) adalah rata-rata penjualan minuman beralkohol per transaksi di Iowa, Amerika Serikat (teruji dengan hypothesis testing two sample independent).
5. \$89.26 (Januari 2018) dan \$106.83 (Januari 2022) adalah rata-rata penjualan minuman beralkohol per transaksi di Iowa, Amerika Serikat (teruji dengan hypothesis testing two sample independent).
6. \$16.04 million (Januari 2018) dan \$18.09 million (Januari 2022) adalah GMV minuman beralkohol di Iowa, Amerika Serikat.
7. Rata-rata harga minuman beralkohol pada bulan Januari 2018 lebih rendah daripada 2022 di Iowa, Amerika Serikat (teruji dengan hypothesis testing two sample independent).

## Benang merah

Seandainya terjadi krisis, perusahaan tidak perlu khawatir terjadi perubahan permintaan dalam pasar. Baik pangsa pasar yang dilayani per kota (Top 1-3), per ukuran botol (Top 1-5), per kategori minuman beralkohol (Top 1-5) tidak berubah. Bahkan, rata-rata nilai transaksi meningkat pada puncak krisis.

Selain itu, jika keadaan mengharuskan perusahaan melakukan downsizing inventory, hanya dengan fokus pada Top 1-5 ukuran botol, Top 1-5 kategori minuman beralkohol, perusahaan dapat melayani 43% pasar (Januari 2022).

## Rekomendasi

Jika perusahaan mengikuti rekomendasi Data Analyst, maka perusahaan dapat melayani pasar dengan nilai sebesar \$679,000 pada bulan Januari 2022.

Nilai pasar dihitung sebagai berikut:
```
GMV minuman beralkohol di Iowa, Amerika Serikat 
* pangsa pasar yang dilayani, yaitu kota Des Moines 
* pangsa pasar yang dilayani berdasarkan ukuran botol minuman beralkohol 
* pangsa pasar yang dilayani berdasarkan kategori minuman beralkohol
```

Rekomendasi Data Analyst adalah sebagai berikut:

Disclaimer: perusahaan membuka toko pada tahun 2022 (bukan tahun 2023, prediksi bukan scope dari Data Analyst saat ini)

1. Kota yang perusahaan harus pilih adalah Des Moines jika ingin berada pada 8.55% pasar.
2. Ukuran botol minuman beralkohol yang harus disediakan perusahaan adalah 750 ml, 1750 ml, 375 ml dan 1000 ml jika ingin melayani 89.23% pasar.
3. Kategori minuman beralkohol yang harus disediakan perusahaan adalah American Vodkas, Canadian Whiskies, Spiced Rum, Straight Bourbon Whiskies, dan Whiskey Liquer jika ingin melayani 49.2% pasar.
4. Promo yang harus dijalankan untuk menarik pelanggan dari kompetitor adalah "Min. Belanja \$100" atau "Min. Volume 6 Liter"